# Simulate exoplanet yield from TESS

The purpose of this code is to simulate the exoplanet yield from the TESS Mission. We do this by taking the various fields that TESS observes and, using a galaxy model, put planets orbiting the stars and see whether we can detect that planet.

In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt
import glob

%matplotlib inline

msun = 1.9891E30
rsun = 695500000.
G = 6.67384E-11
AU = 149597870700.


lets read our galaxt model files

In [2]:
import matplotlib
matplotlib.style.use('ggplot')

names = ['Dist','Mv','CL','Typ','LTef','logg','Age',
         'Mass','BV','UB','VI','VK','V','FeH',
         'l','b','Av','Mbol']

galmodfiles = glob.glob('../data/besmod*.csv')

thisGalmodidx = 0

galmodarea = np.genfromtxt('bess_reas.txt',usecols=2)[thisGalmodidx]

intial_q = pd.read_csv(galmodfiles[thisGalmodidx], skiprows=2, names=names)


In [3]:
intial_q['isMdwarf'] = pd.Series((intial_q.CL == 5) & (intial_q.Typ >= 7.), name='isMdwarf' )
intial_q['I'] = pd.Series(-1. * (intial_q.VI - intial_q.V), name='I')
intial_q['Teff'] = pd.Series(10**intial_q.LTef , name='Teff')

g = 10**intial_q.logg * 0.01
intial_q['Radius'] = pd.Series(np.sqrt(G*intial_q.Mass*msun / g) / rsun, name='Radius')

we previously got the besancon models, they are in the directory ../data/
We also saved the areas for each field in bess_reas.txt

We are doing this in a monte carlo fashion. Our outer loop is each field. The row closest to the equator is easiest because there is no overlap.
We saved a few functions in occSimFuncs.py

In [10]:
from occSimFuncs import (Fressin_select, Dressing_select, per2ars, 
                         get_duration, TESS_noise_1h, nearly_equal, get_transit_depth)


consts =  {'obslen': 75, #days
            'sigma_threshold': 10.,
           'simsize': 8, #size of the galmod field in sq deg
            'full_fov': True, # if true do whole 24x24deg ccd
          }

#make the catalog equal to the full fov area
if consts['full_fov'] & (consts['simsize'] < galmodarea):
    multiple = galmodarea / consts['simsize']
    numstars = int(intial_q.shape[0] * multiple)
    rows = np.random.choice(intial_q.index.values, size=numstars)
    newq = intial_q.ix[rows]
    
    q = newq.set_index(np.arange(newq.shape[0]))
    
elif (simsize > galmodarea):
    raise('Galmod area is too small!')

else:
    q = intial_q



#some planet parameters we will need later
q['cosi'] = pd.Series(np.random.random(size=q.shape[0]),name='cosi')

q['noise_level'] = TESS_noise_1h(q.I)

#reload(occSimFuncs)

draw a bunch of planest and accociate them with each star

In [16]:
#draw a bunch of planets, we will run Dressing for cool stars and Fressin for more massive
mstar_planets = Dressing_select(numstars)
ms_planets = Fressin_select(numstars)

q['planetRadius'] = pd.Series(np.where(q.isMdwarf,mstar_planets[0],ms_planets[0]), name='planetRadius')
q['planetPeriod'] = pd.Series(np.where(q.isMdwarf,mstar_planets[1],ms_planets[1]), name='planetPeriod')


# for i, thisStar in enumerate(q.isMdwarf):
#     if thisStar == True:
#         q.loc[i,'planetRadius'], q.loc[i,'planetPeriod'] = Dressing_select()
#     else:
#         q.loc[i,'planetRadius'], q.loc[i,'planetPeriod'] = Fressin_select()

q['Ntransits'] = np.floor(consts['obslen'] / q.planetPeriod)
q['ars'] = per2ars(q.planetPeriod,q.Mass,q.Radius)
q['duration'] = get_duration(q.planetPeriod,q.ars,b=q.impact)
q['duration_correction'] = np.sqrt(q.duration) # correction for CDPP because transit dur != 1 hour
q['transit_depth']  = get_transit_depth(q.planetRadius,q.Radius)

now lets see if those planets are detected

In [17]:
q['needed_for_detection'] = (q.transit_depth * q.duration_correction *
            np.sqrt(q.Ntransits)) / consts['sigma_threshold']
q['has_planets']  = q.planetRadius > 0.0

In [19]:
q['detected'] = (q.noise_level < q.needed_for_detection) & (q.Ntransits > 1) & (q.planetRadius > 0.0) & q.has_planets

In [20]:
total_planets = (q.ars**-1)[q.detected]
print('total planets = {}'.format(np.sum(total_planets)))

total planets = 37.0262490916
